# Painel Fotovoltaico
### Autora: Luana Dias Costa Lucas <br> Empresa : WP SOLUÇÕES INDUSTRIAIS INTELIGENTES LTDA


## RELATÓRIO LIMPEZA

Será considerado que cada inversor é responsável por um conjunto de paineis, e pelo id dos inversores conseguimos saber quando que os paineis foram limpos 

São duas variaveis a serem considerada, se a ultima limpeza foi a mais tempo do que o recomendado (6 meses), ou se choveu mais do que 600mm de chuva entre o periodo da ultima limpeza e a data atual

In [2]:
#Bibliotecas
from ipynb.fs.full.Helper import *
from datetime import date, datetime, time
from dateutil.relativedelta import relativedelta
from fpdf import FPDF
import pandas as pd
import os

#cleaning = pd.read_csv('archive/Plant_1_Generation_Data_Cleaning.csv') 
#plant = '1'

In [ ]:
# Escolha da central solar
plant = whichPlant()        
if plant == '1':
    path = 'archive/Plant_1/'
else:
    path = 'archive/Plant_2/'
cleaning = pd.read_csv(path + 'Generation_Data_Cleaning.csv')

In [3]:
# Analisa a necessidade de limpeza por inversor de acordo com a data da ultima e da quantidade de chuva que caiu durante o periodo entre a ultima limpeza e a data de hoje
cleaning_updated = cleaning.sort_values(by=['LAST_CLEANING']).reset_index().drop(columns = ['index'])
x = len(cleaning_updated.index)
dataLastCleaning = datetime.today()
today = datetime.today()

for i in range(x):
    lastCleaning = datetime.strptime(cleaning_updated['LAST_CLEANING'][i], '%Y-%m-%d').date()
    gap = cleaning_updated['GAP_MONTHS'][i]
    if lastCleaning + relativedelta(months=+gap) <= date.today():
        cleaning_updated.loc[i, 'CLEANING_REQUIRED'] = 'Sim'
    else:
        if dataLastCleaning != datetime.combine(lastCleaning, time(0,0,0)):
            dataLastCleaning = datetime.combine(lastCleaning, time(0,0,0))
            amountRain = fetchWeatherData(dataLastCleaning, today, -27.58818240596342, -48.56317149277192, 'prcp')
        # Tendo que a quantidade de 600mm de chuva caida durante o periodo seja o suficiente para precisar limpar
        if amountRain >= 600:
            cleaning_updated.loc[i, 'CLEANING_REQUIRED'] = 'Sim'
        else:
            cleaning_updated.loc[i, 'CLEANING_REQUIRED'] = 'Nao'

cleaning_updated.to_csv(path + 'Generation_Data_Cleaning.csv', index=False)


In [43]:
cleaning_required = cleaning_updated.drop(cleaning_updated[cleaning_updated['CLEANING_REQUIRED'] != 'Sim'].index)
cleaning_required = cleaning_required.drop(columns = ['CLEANING_REQUIRED', 'GAP_MONTHS']).reset_index()

In [44]:
pdf = FPDF()
# Set left margin
pdf.set_left_margin(0)
pdf.add_page()
pdf.set_font('Arial', 'B', 14)

## Title
pdf.cell(0,15,'Conjunto de paineis que precisam de limpeza - ' + date.today().strftime("%d/%m/%Y"), align='C')
pdf.ln(20)
pdf.set_font('Arial', '', 10)
pdf.cell(10)

cleaning_req_pdf = cleaning_required.drop(columns = ['index']).reset_index()
x = len(cleaning_req_pdf.index)

pdf.cell(0,0,'São ao total ' + str(x) + ' painéis que precisam de limpeza, abaixo está a lista com a Source Key e a data da última limpeza' , align='L')
pdf.ln(10)

spreadsheet_df_to_pdf(pdf, cleaning_req_pdf, 2, 40, True)

dataHoje = date.today().strftime("%d%m%Y")

if not os.path.exists('Relatorios/Limpeza/Plant' + plant):
   os.makedirs('Relatorios/Limpeza/Plant' + plant)

pdf.output('Relatorios/Limpeza/Plant' + plant + '/Conjunto de paineis que precisam de limpeza ' + dataHoje + '.pdf', 'F')


''